In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 4.6MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 4.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin,tpe, STATUS_OK
import eli5
from eli5.sklearn import PermutationImportance
import xgboost as xgb

In [4]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two


In [5]:
df =pd.read_hdf('data/car.h5')
df.shape
df = df[df['price_currency'] != 'EUR']
df.shape

(106290, 155)

In [0]:
SUFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  factorized_valus = df[feat].factorize()[0]
  if SUFIX_CAT in feat:
    df[feat] = factorized_valus
  else:
    df[feat + SUFIX_CAT] = factorized_valus

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else  int(x.split(' ')[0]) )
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else  int(x.split('cm')[0].replace(" ", '')) )

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  #model = DecisionTreeRegressor(max_depth = 5)
  scores = cross_val_score(model, X, y, cv =3, 
                          scoring = 'neg_mean_absolute_error' )
  return np.mean(scores), np.std(scores)

In [0]:
#feats3 = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa__cat','seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat','feature_czujniki-parkowania-przednie__cat','feature_asystent-pasa-ruchu__cat','feature_regulowane-zawieszenie__cat','feature_system-start-stop__cat','feature_światła-led__cat']
#run_model(xgb.XGBRegressor(**xgb_params), feats3)


In [12]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa__cat','seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat','feature_czujniki-parkowania-przednie__cat','feature_asystent-pasa-ruchu__cat','feature_regulowane-zawieszenie__cat','feature_system-start-stop__cat','feature_światła-led__cat']

#XGBoost
xgb_params= {
    'max_depth' : 5, 
    'n_estimators':50, 
    'lerning_rate' :0.1,
    'seed' : 0
}
run_model(xgb.XGBRegressor(**xgb_params), feats)

[18:38:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:38:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:38:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9644.639791395812, 74.61610121163075)

In [25]:
def obj_func(params):
  print("training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
  return {'loss':np.abs(mean_mae), 'status':STATUS_OK}

  #space
xgb_reg_params ={
    'lerning_rate':  hp.choice('lerning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth': hp.choice('max_depth', np.arange(1, 16, 1, dtype =int)),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsapmle_bytree', 0.5, 1, 0.05),
    'objective': 'reg:squarederror',
    'n_estimators': 100,
    'seed' : 0
  }

best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals= 25)
best

training with params: 
{'colsample_bytree': 0.5, 'lerning_rate': 0.25, 'max_depth': 2, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
training with params: 
{'colsample_bytree': 0.9500000000000001, 'lerning_rate': 0.05, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.5}
training with params: 
{'colsample_bytree': 0.8, 'lerning_rate': 0.1, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
training with params: 
{'colsample_bytree': 0.8500000000000001, 'lerning_rate': 0.15000000000000002, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
training with params: 
{'colsample_bytree': 0.75, 'lerning_rate': 0.25, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
training with params: 
{'colsample_bytree': 0.8500000000000001, 'lerni

{'colsapmle_bytree': 1.0,
 'lerning_rate': 3,
 'max_depth': 10,
 'subsample': 0.9500000000000001}

In [0]:
#feats3 = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa__cat','seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat','feature_czujniki-parkowania-przednie__cat','feature_asystent-pasa-ruchu__cat','feature_regulowane-zawieszenie__cat','feature_system-start-stop__cat','feature_światła-led__cat']
#run_model(xgb.XGBRegressor(**xgb_params), feats3)
